# ARXIV Keyword Project

In diesem Tutorial Project analysieren wir, zu welchen Themen in der Informatik in letzter Zeit vermehrt veröffentlicht werden.

Erstmal ein paar imports und ein logger setup:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import webbrowser
from tqdm.notebook import tqdm


import arxiv, logging, keybert
from collections import Counter
from wordcloud import WordCloud


In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
url = 'https://arxiv.org/search/cs?query=peer+kr%C3%B6ger&searchtype=all&abstracts=show&order=-announced_date_first&size=50'

# Open arxiv website of prof. peer kröger
webbrowser.open_new_tab(url)

True

## 1. Daten sammeln

Wir holen uns erstmal die Meta Daten zu den letzten 1000 Papern aus der Computer Science Kategorie. Hierzu nutzen wir die [arxiv python library](https://pypi.org/project/arxiv/), welche wiederrum auf die offizielle arxiv API zugreift. Die Ergebnisse werden dann vom neusten zum ältesten Einreichdatum sortiert.

In [4]:
search = arxiv.Search( 
    query='cat:cs.*',
    sort_by = arxiv.SortCriterion.SubmittedDate,
    max_results = 200,
    sort_order = arxiv.SortOrder.Descending
)

Durch den Befehl search.results() werden die Daten tatsächlich angefragt und dann in einer FOR-Schleife in eine Liste von Dictionaries mit den Meta Daten, die wir weiterhin benötigen, eingefügt.

In [5]:
results = []
for paper in search.results():
    meta_data = {
        'title' : paper.title,
        'abstract' : paper.summary,
        'authors' : paper.authors,
        'date_published' : paper.published
    }
    results.append(meta_data)

INFO:arxiv.arxiv:Requesting 100 results at offset 0
INFO:arxiv.arxiv:Requesting page of results
INFO:arxiv.arxiv:Got first page; 200 of 200 results available
INFO:arxiv.arxiv:Requesting 100 results at offset 10
INFO:arxiv.arxiv:Sleeping for 2.997644 seconds
INFO:arxiv.arxiv:Requesting page of results
INFO:arxiv.arxiv:Requesting 100 results at offset 100
INFO:arxiv.arxiv:Sleeping for 2.995267 seconds
INFO:arxiv.arxiv:Requesting page of results


Nun laden wir die Daten in ein pandas DataFrame und setzten den Index als den Timestamp an dem das Paper veröffentlicht wurde.

In [6]:
results_df = pd.DataFrame(results)
results_df.set_index(results_df['date_published'], inplace=True)
results_df

,title,abstract,authors,date_published
date_published,,,,
2024-08-16 17:59:46+00:00,Accelerating Giant Impact Simulations with Mac...,Constraining planet formation models based on ...,"[Caleb Lammers, Miles Cranmer, Sam Hadden, Shi...",2024-08-16 17:59:46+00:00
2024-08-16 17:57:01+00:00,xGen-MM (BLIP-3): A Family of Open Large Multi...,This report introduces xGen-MM (also known as ...,"[Le Xue, Manli Shu, Anas Awadalla, Jun Wang, A...",2024-08-16 17:57:01+00:00
2024-08-16 17:55:38+00:00,SAM2-UNet: Segment Anything 2 Makes Strong Enc...,Image segmentation plays an important role in ...,"[Xinyu Xiong, Zihuang Wu, Shuangyi Tan, Wenxue...",2024-08-16 17:55:38+00:00
2024-08-16 17:54:09+00:00,PEDAL: Enhancing Greedy Decoding with Large La...,Self-ensembling techniques with diverse reason...,[Sumanth Prabhu],2024-08-16 17:54:09+00:00
2024-08-16 17:52:22+00:00,A Hassle-free Algorithm for Private Learning i...,The state-of-the-art for training on-device la...,"[H. Brendan McMahan, Zheng Xu, Yanxiang Zhang]",2024-08-16 17:52:22+00:00
...,...,...,...,...
2024-08-16 04:57:54+00:00,Detecting Unsuccessful Students in Cybersecuri...,This full paper in the research track evaluate...,"[Valdemar Švábenský, Kristián Tkáčik, Aubrey B...",2024-08-16 04:57:54+00:00
2024-08-16 04:57:21+00:00,Privacy-Preserving Vision Transformer Using Im...,We propose a novel method for privacy-preservi...,"[Kouki Horio, Kiyoshi Nishikawa, Hitoshi Kiya]",2024-08-16 04:57:21+00:00
2024-08-16 04:56:58+00:00,Study of MRI-compatible Notched Plastic Ultras...,Intra-operative image guidance using magnetic ...,"[Zhanyue Zhao, Haimi Tang, Paulo Carvalho, Cos...",2024-08-16 04:56:58+00:00


# 2. Keywords generieren

Arxiv hat leider keine Keywords im Angebot, also nutzen wir eine Deep Learning Methode um die Keywords aus den jeweiligen Abstracts der Paper zu extrahieren. Hier gibt es netterweise eine python bibliothek, [KeyBERT](https://maartengr.github.io/KeyBERT/), welche ein schon komplett trainiertes Modell direkt bereitstellt, dass laden wir jetzt erstmal in unser Notebook.

In [7]:
model = keybert.KeyBERT()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Als nächstes wenden wir KeyBERT auf den Titel kombiniert mit dem Abstract von jedem Paper an und **speichern keywords bestehend aus ein und zwei Wörtern ab**.

<span style="color:red">Das kann ein bisschen dauern</span>!

In [ ]:
keywords_list = []
for row_i in tqdm(range(results_df.shape[0])):
    row = results_df.iloc[row_i]
    keywords = []
    title_abstract = row['title'] + ' ' + row['abstract'] 
    keywords.append(model.extract_keywords(title_abstract, 
                       keyphrase_ngram_range = (1,1))) # keywords containing one word
    #keywords.append(model.extract_keywords(title_abstract, 
    #                   keyphrase_ngram_range = (1,2))) # keyphrases containing two words
    keywords_list.append(keywords)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
keywords_list[0:1]

Die Keywords werden als eigene Spalte an das DataFrame angehängt.

In [ ]:
results_df['keywords'] = keywords_list

In [ ]:
results_df

Die Keywords-Liste ist jetzt eine Liste aus Listen von Keywords, um jetzt möglichst einfach die Keywords zu zählen, wandeln wir diese in nur eine Liste mit Keywords um mit Hilfe von dieser flatten Funktion:

In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [ ]:
flattened_keywords = flatten(flatten(keywords_list))

In [ ]:
flattened_keywords = [item[0] for item in flattened_keywords]

In [ ]:
flattened_keywords[0:1], flattened_keywords[0:4]

Mit Hilfe von der python *set*-Funktion ist schnell herausgefunden, wie viele einzigartigen Keywords in der gesamten Liste enthalten sind.

In [ ]:
len(flattened_keywords)

In [ ]:
len(set(flattened_keywords))

#### Datensatz aus nur Keywords
Jetzt suchen wir die häufigsten Keywords und speichern sie in einem eigenen DataFrame ab, wir filtern davor Worte die nur einmalig vorkommen raus und sortieren das ganze dann nach der absteigend nach der Häufigkeit.

In [ ]:
keywords_df = pd.DataFrame()
keywords_df['words'] = Counter(flattened_keywords).keys()
keywords_df['numbers'] = Counter(flattened_keywords).values()
frequent_words = keywords_df.loc[keywords_df['numbers'] > 1].sort_values(by='numbers', ascending=False)
frequent_words

# 3. Visualizierung der Daten

Und jetzt können wir direkt aus dem Counter das Ganze als Wordcloud visualisieren.

In [ ]:
wordcloud = WordCloud(background_color="white", 
                      width=2500, 
                      height=1000, 
                      scale=2.0, 
                      min_font_size=6).fit_words(dict(Counter(flattened_keywords))).to_file('arxiv_word_cloud.png')

In [ ]:
plt.imshow(wordcloud)
# plt.savefig('arxiv_word_cloud.png')
plt.show()

# Aufgaben

0. Prüfe, ob es tatsächlich so ist, dass die populärsten Keyphrases nur ein Wort beinhalten? Wenn nicht, wieso tauchen überwiegend einwörtige Keyphrases in der Wordcloud groß auf? 

1. Probiere die WordCloud direkt aus den Titel und/oder den Abstracts der Paper zu erzeugen, ohne das KeyBERT zu nutzen und vergleiche die Wordclouds. Entsprechen die häufigst verwendeten Worte genau den Keywords? Wenn nicht: Was ist die Ursache und wie könnte man diese einfach umgehen? *Hint: Stop Words (auch direkt in der [word cloud bib](https://amueller.github.io/word_cloud/index.html))*

2. Erstelle eine neue Visualizierung der häufigsten Keywords, welche keine WordCloud verwendet.

3. Statt die vorgefertigte *arxiv* libary zu nutzen, beziehe die Daten aus der API nur mit Hilfe der python internen requests Bibliothek. Passt dabei auf die Benutzungs-Hinweise von Arxive, auf der [arxiv API Webseite](https://arxiv.org/help/api/), auf und bitte keine DDOS Attacke auf die Server starten. (Baue also Wartezeiten ein!)

In [ ]:
import requests

4. Bonusaufgabe - Erstellt einen Co-Autoren Graph aus dem Autoren Attribut der Paper und visualsiert diesen. Dieser Artikel könnte helfen https://towardsdatascience.com/visualizing-networks-in-python-d70f4cbeb259